# FMCA interface

### first import modules

In [ ]:
# import seems necessary to not crash matplotlib
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy import linalg as la
import time
import FMCA

### generate data points and samplet tree
(We force NumPy to use column major arrays for performance reasons)

In [ ]:
dim = 2
N = 10000
dtilde = 3
pts = np.array(np.random.randn(dim, N), order='F')
start = time.time()
ST = FMCA.SampletTree(pts, dtilde)
stop = time.time()
print('elapsed time samplet tree construction: ', stop - start, 'sec.')
FMCA.sampletTreeStatistics(ST, pts)
# reorder points according to samplet tree order
spts = pts[:, ST.indices()]

### transform a given set of data vectors into samplet basis and back
(We force NumPy to use column major arrays for performance reasons)

In [ ]:
start = time.time()
data = 2 * np.array(np.random.rand(N,10), order='F') - 1
Tdata = FMCA.sampletTransform(ST, data)
data2 = FMCA.inverseSampletTransform(ST, Tdata)
stop = time.time()
print('elapsed time forward and backward transform: ', stop - start, 'sec.')
print('relative error: ', la.norm(data - data2) / la.norm(data))

### plot a samplet
(Basis elements can be retrieved by inverse transforming unit vectors)

In [ ]:
unit = np.array(np.zeros(N), order='F');
unit[22] = 1
Tunit = FMCA.inverseSampletTransform(ST, unit)
plt.figure(figsize=(8,8))
ax = plt.axes(projection='3d')
ax.scatter(spts[0,:], spts[1,:], Tunit[:,0], c=Tunit[:,0], marker='.', cmap='turbo')
plt.show()

### transform a random signal into the samplet basis
(Basis elements can be retrieved by inverse transforming unit vectors)

In [ ]:
dim = 1
N = 300000
dtilde = 6
pts = 2 * np.pi * np.array(np.random.rand(dim, N), order='F')
ST = FMCA.SampletTree(pts, dtilde)
FMCA.sampletTreeStatistics(ST, pts)
# reorder points according to samplet tree order and make it an array
spts = pts[:, ST.indices()].flatten()

In [ ]:
data = np.sin(10 * spts) * np.cos(2 * spts) + 0.2 * np.sin(80 * spts) * np.cos(12 * spts)
plt.figure(figsize=(15,5))
plt.plot(spts,data)
plt.axis('tight')
plt.title('signal sampled at 100000 random pts')
plt.show()

In [ ]:
Tdata = FMCA.sampletTransform(ST, data)
plt.figure(figsize=(15,5))
plt.plot(Tdata[0:500])
plt.axis('tight')
plt.title('first 500 samplet coefficients')
plt.show()

In [ ]:
dim = 2
N = 100000
dtilde = 14
pts = 2 * np.pi * np.array(np.random.rand(dim, N), order='F')
start = time.time()
ST = FMCA.SampletTree(pts, dtilde)
stop = time.time()
print('time tree setup:  ', stop - start, 'sec.')
cov = FMCA.CovarianceKernel("GAUSSIAN", 2)
start = time.time()
SKC = FMCA.SampletKernelCompressor(ST, cov, pts, 0.8, 1e-6)
stop = time.time()
print('time kernel compression:  ', stop - start, 'sec.')

In [ ]:
import scipy
K = SKC.matrix()
plt.spy(K, markersize=1)
plt.show() 
plt.imshow(np.log(np.abs((K[:2000,:2000]).todense()+1e-16)))
plt.show()

In [ ]:
plt.semilogy(np.sort(K.diagonal()))
plt.show()
print(np.min(K.diagonal()))